In [108]:
import numpy as np
import pandas as pd
from re import sub

In [109]:
df = pd.read_csv('../datasets/raw/movies.csv')
df.head()

,Title,Rating,Year,Month,Certificate,Runtime,Directors,Stars,Genre,Filming_location,Budget,Income,Country_of_origin
0,Avatar: The Way of Water,8.0,2022,December,PG-13,192,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...","Action, Adventure, Fantasy",New Zealand,"$350,000,000","$681,081,686",United States
1,Guillermo del Toro's Pinocchio,7.8,2022,December,PG,117,"Guillermo del Toro, Mark Gustafson","Ewan McGregor, David Bradley, Gregory Mann, Bu...","Animation, Drama, Family",USA,"$35,000,000","$71,614","United States, Mexico, France"
2,Bullet Train,7.3,2022,August,R,127,David Leitch,"Brad Pitt, Joey King, Aaron Taylor Johnson, Br...","Action, Comedy, Thriller",Japan,"$85,900,000","$239,268,602","Japan, United States"
3,The Banshees of Inisherin,8.0,2022,November,R,114,Martin McDonagh,"Colin Farrell, Brendan Gleeson, Kerry Condon, ...","Comedy, Drama",Ireland,Unknown,"$19,720,823","Ireland, United Kingdom, United States"
4,M3gan,NaN,2022,January,PG-13,102,Gerard Johnstone,"Jenna Davis, Amie Donald, Allison Williams, Vi...","Horror, Sci-Fi, Thriller",New Zealand,Unknown,Unknown,United States


In [110]:
df.columns = df.columns.str.lower()

In [111]:
memory_before = df.memory_usage(deep=True).sum()
memory_before

1622481

In [112]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              2000 non-null   object 
 1   rating             1998 non-null   float64
 2   year               2000 non-null   int64  
 3   month              2000 non-null   object 
 4   certificate        1966 non-null   object 
 5   runtime            2000 non-null   object 
 6   directors          2000 non-null   object 
 7   stars              2000 non-null   object 
 8   genre              2000 non-null   object 
 9   filming_location   2000 non-null   object 
 10  budget             2000 non-null   object 
 11  income             2000 non-null   object 
 12  country_of_origin  2000 non-null   object 
dtypes: float64(1), int64(1), object(11)
memory usage: 1.5 MB


In [113]:
df.fillna(
    {
        'certificate': df.certificate.mode()[0],
        'rating': df.rating.median()
    }, inplace=True
)

In [114]:
for column in df.columns:
    print(f"{column}: {len(df[column].unique())}")

title: 1989
rating: 62
year: 20
month: 14
certificate: 12
runtime: 113
directors: 1082
stars: 1990
genre: 244
filming_location: 97
budget: 305
income: 1855
country_of_origin: 406


In [115]:
df.year = df.year.astype('uint16')

In [116]:
df.month.unique()

array(['December', 'August', 'November', 'January', 'October', 'March',
       'September', 'May', 'April', 'July', 'June', 'February', '2014',
       '2008'], dtype=object)

In [117]:
df.loc[df.query('month == "2014" | month == "2008"').index, 'month'] = df.month.mode()[0]
df.month = df.month.astype('category')

In [118]:
df.certificate.unique()

array(['PG-13', 'PG', 'R', 'TV-14', 'TV-MA', 'TV-PG', 'TV-Y7',
       'Not Rated', 'NC-17', 'TV-G', 'Unrated', 'G'], dtype=object)

In [119]:
df.loc[(df.query('certificate == "Not Rated" | certificate == "Unrated"')).index, 'certificate'] = 'Unrated'
df.certificate = df.certificate.astype('category')

In [120]:
df.loc[(df.query('runtime == "Unknown"')).index, 'runtime'] = df.drop(index = df.query('runtime == "Unknown"').index).runtime.astype('int64').median()
df.runtime = df.runtime.astype('uint8')

In [121]:
df.filming_location = df.filming_location.astype('category')

Todos os valores cambiais são do dia 16/02
- $: Dolar Americano
- €: Euro
- NOK\xa0: Coroa Norueguesa Franco
- SEK\xa0: Coroa Sueca
- A$: Dólar Austráliano
- ₹: Rupia Indiana
- CA$: Dólar Canadense
- CN¥: Yuan Chinês
- £: Libra Esterlina
- ₩: Won Sul-Coreano
- ¥: Iene Japonês
- DKK\xa0: Coroa Dinamarquesa

In [122]:
simb = df.budget.str.replace(r'[0-9|,]', '', regex=True).unique()
conversion = {'$': 1.0,'€': 1.0692577, 'NOK\xa0': 0.097486342, 'SEK\xa0': 0.09551, 'A$': 0.68763434, '₹': 0.01209, 'CA$': 0.74523201, 'CN¥': 0.14572 ,'£': 1.1987, '₩': 0.00077412484, '¥': 0.0074468957, 'DKK\xa0': 0.14329}

In [123]:
def convert_currency(value):
    if value == 'Unknown':
        return value
    else:
        for item in simb:
            if item in value:
                return float(sub(r'[^\d.]', '', value)) * conversion[item]
            
df.budget = df.budget.apply(convert_currency)
del convert_currency

In [124]:
df.income = df.income.str.lstrip('$').replace(to_replace=r'[,]', value='', regex=True)

In [125]:
df.loc[(df.query('income == "Unknown"')).index, 'income'] = df.drop(index = df.query('income == "Unknown"').index).income.astype('float64').median()
df.loc[(df.query('budget == "Unknown"')).index, 'budget'] = df.drop(index = df.query('budget == "Unknown"').index).budget.astype('float64').median()

In [126]:
df.income = df.income.astype('float64')
df.budget = df.budget.astype('float64')

In [127]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   title              2000 non-null   object  
 1   rating             2000 non-null   float64 
 2   year               2000 non-null   uint16  
 3   month              2000 non-null   category
 4   certificate        2000 non-null   category
 5   runtime            2000 non-null   uint8   
 6   directors          2000 non-null   object  
 7   stars              2000 non-null   object  
 8   genre              2000 non-null   object  
 9   filming_location   2000 non-null   category
 10  budget             2000 non-null   float64 
 11  income             2000 non-null   float64 
 12  country_of_origin  2000 non-null   object  
dtypes: category(3), float64(3), object(5), uint16(1), uint8(1)
memory usage: 878.0 KB


In [128]:
memory_after = df.memory_usage(deep=True).sum()
memory_after

899104

In [129]:
print (f'{memory_after/memory_before*100: .2f}% ganho de memória')

 55.42% ganho de memória


In [130]:
df.to_pickle('../datasets/processed/movies.pkl')